In [46]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
import joblib

In [47]:
import os
os.getcwd()

'/Users/mschonhofer/PycharmProjects/RP2/backend/app'

In [48]:
df = pd.read_csv("../../data/Data.csv")
df.head()

,Department,Gender,HSC,SSC,Income,Hometown,Computer,Preparation,Gaming,Attendance,Job,English,Extra,Semester,Last,Overall
0,Business Administration,Male,4.17,4.84,"Low (Below 15,000)",Village,3,More than 3 Hours,0-1 Hour,80%-100%,No,3,Yes,6th,3.220,3.350
1,Business Administration,Female,4.92,5.00,"Upper middle (30,000-50,000)",City,3,0-1 Hour,0-1 Hour,80%-100%,No,3,Yes,7th,3.467,3.467
2,Business Administration,Male,5.00,4.83,"Lower middle (15,000-30,000)",Village,3,0-1 Hour,More than 3 Hours,80%-100%,No,4,Yes,3rd,4.000,3.720
3,Business Administration,Male,4.00,4.50,"High (Above 50,000)",City,5,More than 3 Hours,More than 3 Hours,80%-100%,No,5,Yes,4th,3.800,3.750
4,Business Administration,Female,2.19,3.17,"Lower middle (15,000-30,000)",Village,3,0-1 Hour,2-3 Hours,80%-100%,No,3,Yes,4th,3.940,3.940


In [49]:
df.columns.tolist()

['Department',
 'Gender',
 'HSC',
 'SSC',
 'Income',
 'Hometown',
 'Computer',
 'Preparation',
 'Gaming',
 'Attendance',
 'Job',
 'English',
 'Extra',
 'Semester',
 'Last',
 'Overall']

In [64]:
df.isna().sum()

Department         0
Gender             0
HSC                0
SSC                0
Income             0
Hometown           0
Computer           0
Preparation        0
Gaming             0
Attendance         0
Job                0
English            0
Extra              0
Semester           0
Last               0
Overall            0
target_stem        0
Income_clean       0
semester_num       0
semester_norm      0
academic_score     0
self_discipline    0
income_norm        0
hometown_norm      0
gender_norm        0
dtype: int64

In [51]:
df["target_stem"] = df["Department"].isin([
    "Computer Science and Engineering"
]).astype(int)

df["target_stem"].value_counts()

target_stem
1    443
0     50
Name: count, dtype: int64

In [52]:
df["Income_clean"] = (
    df["Income"]
    .astype(str)
    .str.strip()
    .str.replace("\u00a0", " ", regex=False)
    .str.replace(r"\s+", " ", regex=True)
)

In [53]:
prep_map = {
    "0-1 Hour": 0.2,
    "2-3 Hours": 0.6,
    "More than 3 Hours": 0.9,
}

gaming_map = {
    "0-1 Hour": 0.85,
    "2-3 Hours": 0.55,
    "More than 3 Hours": 0.25,
}

attendance_map = {
    "Below 40%": 0.15,
    "40%-59%": 0.4,
    "60%-79%": 0.7,
    "80%-100%": 0.95,
}

yesno_map = {"Yes": 1.0, "No": 0.0}

income_map = {
    "Low (Below 15,000)": 0.2,
    "Lower middle (15,000-30,000)": 0.45,
    "Upper middle (30,000-50,000)": 0.7,
    "High (Above 50,000)": 0.9,
}

hometown_map = {
    "Village": 0.3,
    "Town": 0.5,
    "City": 0.7,
    "Other": 0.6,
}

gender_map = {
    "Male": 1.0,
    "Female": 0.0,
    "Other": 0.5,
}

In [54]:
for col in ["Preparation", "Attendance", "Gaming", "Job", "Extra"]:
    df[col] = (
        df[col]
        .astype(str)
        .str.strip()
        .str.replace("\u00a0", " ", regex=False)
        .str.replace(r"\s+", " ", regex=True)
    )

In [55]:
df["semester_num"] = (
    df["Semester"]
    .str.extract(r"(\d)")
    .astype(float)
)

df["semester_norm"] = df["semester_num"] / df["semester_num"].max()

In [56]:
df["academic_score"] = (
    0.25 * df["SSC"] +
    0.25 * df["HSC"] +
    0.25 * df["Last"] +
    0.25 * df["Overall"]
)

df["academic_score"] /= df["academic_score"].max()

In [57]:
df["self_discipline"] = (
    0.30 * df["Preparation"].map(prep_map) +
    0.25 * df["Attendance"].map(attendance_map) +
    0.20 * df["Gaming"].map(gaming_map) +
    0.10 * (1 - df["Job"].map(yesno_map)) +
    0.15 * df["Extra"].map(yesno_map)
)

In [58]:
df["income_norm"] = df["Income_clean"].map(income_map)

df["hometown_norm"] = (
    df["Hometown"].astype(str).str.strip().str.replace("\u00a0", " ", regex=False)
)
df["hometown_norm"] = df["hometown_norm"].map(hometown_map)

df["gender_norm"] = (
    df["Gender"].astype(str).str.strip().str.replace("\u00a0", " ", regex=False)
)
df["gender_norm"] = df["gender_norm"].map(gender_map)

In [59]:
df[["income_norm", "self_discipline", "hometown_norm", "gender_norm"]].isna().sum()

income_norm        0
self_discipline    0
hometown_norm      0
gender_norm        0
dtype: int64

In [65]:
df["computer_skill"] = df["Computer"].astype(float) / 5
df["english_skill"] = df["English"].astype(float) / 5

# jeśli SSC/HSC/Last/Overall są na skali 0–5 to academic_score już ok
# jeśli na 0–100, nadal OK bo dzielisz przez max

FEATURES = [
    "self_discipline",
    "academic_score",
    "computer_skill",
    "english_skill",
    "income_norm",
    "hometown_norm",
    "gender_norm",
    "semester_norm",
]

X = df[FEATURES]
y = df["target_stem"]

In [66]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [73]:
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(
        max_iter=2000,
        class_weight="balanced",
        solver="lbfgs"
    ))
])

pipe.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('scaler', ...), ('clf', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"copy copy: bool, default=TrueIf False, try to avoid a copy and do inplace scaling instead.This is not guaranteed to always work inplace; e.g. if the data isnot a NumPy array or scipy.sparse CSR matrix, a copy may still bereturned.",True
,"with_mean with_mean: bool, default=TrueIf True, center the data before scaling.This does not work (and will raise an exception) when attempted onsparse matrices, because centering them entails building a densematrix which in common use cases is likely to be too large to fit inmemory.",True
,"with_std with_std: bool, default=TrueIf True, scale the data to unit variance (or equivalently,unit standard deviation).",True
,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work

In [72]:
from sklearn.metrics import confusion_matrix

pred = pipe.predict(X_test)
proba = pipe.predict_proba(X_test)[:, 1]

print("Accuracy:", accuracy_score(y_test, pred))
print("ROC-AUC:", roc_auc_score(y_test, proba))
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred, digits=3))

Accuracy: 0.8888888888888888
ROC-AUC: 0.8325842696629212
[[ 0 10]
 [ 1 88]]
              precision    recall  f1-score   support

           0      0.000     0.000     0.000        10
           1      0.898     0.989     0.941        89

    accuracy                          0.889        99
   macro avg      0.449     0.494     0.471        99
weighted avg      0.807     0.889     0.846        99



In [70]:
coefs = pd.Series(
    pipe.named_steps["clf"].coef_[0],
    index=FEATURES
).sort_values(ascending=False)

coefs

income_norm        0.480187
computer_skill     0.395016
english_skill      0.230360
gender_norm        0.117256
semester_norm      0.094502
academic_score     0.091826
hometown_norm     -0.401480
self_discipline   -0.941031
dtype: float64

In [76]:
joblib.dump(pipe, "../models/stem_model.joblib")

['../app/models/stem_model.joblib']